In [1]:
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

from cvxopt import matrix, solvers
import cvxpy

## Output in Weka format

In [332]:
import os
def predict_for_test(test, predict, probability, path):
    with open(path, 'w') as f:
        f.write("=== Predictions on test data ===\n")
        f.write(" inst#     actual  predicted error prediction\n")
        for i in range(len(test)):
            string = [str(i + 1)]
            if test[i] == 1:
                string.append("1:positive")
            else:
                string.append("2:negative")
            if predict[i] == 1:
                string.append("1:positive")
            else:
                string.append("2:negative")
            if test[i] == predict[i]:
                string.append(" " * 5)
            else:
                string.append(" " * 2 + "+" + " " * 2)
            if predict[i] == 1:
                string.append(str(probability[i][1]))
            else:
                string.append(str(probability[i][0]))
            string = " ".join(string) + "\n"
            f.write(string)   
    

## Load data

In [2]:
import arff

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#import data
def importData(path):
    dataset = arff.load(open(path, 'rb'))
    data = np.array(dataset['data'])
    #print data[:10]

    #extract features and labels
    features = []
    labels = []
    for d in data:
        f = []
        for i in range(len(d) - 1):
            num = float(d[i])
            if int(num) == num:
                num = int(num)
            f.append(num)
        features.append(f)

        if d[-1] == "positive":
            labels.append(1)
        else:
            labels.append(0)
    return np.asarray(features), np.asarray(labels)

In [77]:
#import training data and test data
train_datapath = "../myclassify/qa.train.arff"
test_datapath = "../myclassify/qa.test.arff"

X_train, y_train = importData(train_datapath)
X_test, y_test = importData(test_datapath)

## Train classifier

In [314]:
import sklearn.linear_model

# Jacana features only
clf = linear_model.LogisticRegression(C=0.01,
#                                          penalty="l1"
                                        )
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Compute metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

             precision    recall  f1-score   support

          0       0.84      0.99      0.91      1233
          1       0.82      0.17      0.28       284

avg / total       0.83      0.84      0.79      1517

[[1222   11]
 [ 235   49]]
Accuracy: 0.837837837838


---
## Word embedding vectors

In [278]:
# Map each word to an index
ndim = 300
glove_path = "../data/glove_embeddings/glove.6B.{}d.txt".format(ndim)
with open(glove_path, "rb") as lines:
    w2idx = {line.split()[0].decode("utf-8"): i for i, line in enumerate(lines)}

In [279]:
vectors = np.empty((len(w2idx), ndim), dtype=np.float)
with open(glove_path, "rb") as lines:
    for i, line in enumerate(lines):
        vectors[i] = np.asarray(map(float, line.split()[1:]))

In [281]:
import string
from nltk.corpus import stopwords

words_to_exclude = frozenset(string.punctuation) | frozenset(["..", "..."])
words_to_exclude |= frozenset(stopwords.words("english"))

---
Get vector indicating which example belongs to which question group


In [110]:
def get_QA_group_count(infile):        
    line = infile.readline().strip()
    if line == "":
        return None
    
    if not line.startswith("<QApairs"):
        raise Exception("Invalid data format: {}<-----".format(line))
    
#     print(line)
    sentence_count = 0
    while not line.strip().startswith("</QApairs"):
        line = infile.readline().replace('\t', ' ')                    
        if line.strip().lower().startswith("<positive") or line.strip().lower().startswith("<negative"):
            sentence_count += 1
    
    return sentence_count

def get_QA_group_indicators(filepath):    
    with open(filepath) as infile:
        indicators = []
        qn_number = 0
        while infile:
            count = get_QA_group_count(infile)

            # Check for EOF
            if count == None:
#                 print("Reached EOF at qn {}".format(qn_number))
                break

            if count > 0:
                indicators += ([qn_number] * count)
                qn_number += 1
        
    return np.asarray(indicators)

---
Compute weight matrix $W$ for each set of question


In [74]:
# Get a question and its candidate answers
def get_QA_group(infile):
    question = []
    answers = []
    line = infile.readline().strip()
    if line == "":
        return None
    
    if not line.startswith("<QApairs"):
        raise Exception("Invalid data format: {}<-----".format(line))
    
#     print(line.strip())
    
    while not line.strip().startswith("</QApairs"):
        line = infile.readline().replace('\t', ' ')
        if line.strip().lower().startswith("<question"):
            line = infile.readline().replace('\t', ' ')
            question.append(line.strip())
        elif line.strip().lower().startswith("<positive"):
            line = infile.readline().replace('\t', ' ')
            answers.append(("positive", line.strip()))
        elif line.strip().lower().startswith("<negative"):
            line = infile.readline().replace('\t', ' ')
            answers.append(("negative", line.strip()))
    
    return {"question": question, "answers": answers}                

In [35]:
from nltk.tokenize import WhitespaceTokenizer

def extract_vector(sentence, exclude, w2idx, wordvectors):
    """Compute the vector for a sentence by averaging the words in the sentence that has word embeddings"""
    # Tokenize sentence
    splitter = WhitespaceTokenizer()
    tokens = splitter.tokenize(sentence)    
    # Remove stopwords and punctuation
    words = [t.lower() for t in tokens if t.lower() not in exclude ]
    
    # If we cannot find any words, we can consider returning a vector of 0
    # and set the resulting cosine similarity to 0 otherwise will result in nan
    # because cosine similarity will divide by 0.
    assert(len(words) > 0)
            
    # Average words in sentence that are in word matrix
    try:
        avg_vec = np.mean([wordvectors[w2idx[w]] for w in words if w in w2idx ] 
                                                 or [np.zeros(wordvectors.shape[1])], 
                           axis=0)
        if not np.any(avg_vec):
            print("Tokens cannot be found: {}".format(words))
        assert(np.any(avg_vec))
        return avg_vec
    except UnicodeDecodeError:
        print(line.strip())
        raise

In [63]:
def compute_pairwise_distance_matrix(X, k):
    """Compute pairwise distances between each point in X
    and its k-nearest neighbors."""

    from scipy.spatial import KDTree
    kdtree = KDTree(X)
    A = np.zeros((X.shape[0], X.shape[0]), dtype=np.float)
    for i, x in enumerate(X):
        distances, idxs = kdtree.query(x, k+1)  # k+1 as one pt is the pt itself.
        for d, j in zip(distances, idxs):
            A[i, j] = d**2  # Store squared euclidean distance

    return A

In [320]:
import scipy as sp

# Compute the scores/features for a dataset
def get_weight_matrix(input_file, n_neighbors=5, sigma=1.0, eps=0.0001):
    """Compute weight matrix for question and answer sentences 
    """
    with open(input_file) as infile:
        num_questions = 0
        while infile:
            group = get_QA_group(infile)

            # Check for EOF
            if group is None:
                break
                
            # Extract question vector
            question = group["question"]
            qvec = extract_vector(question[0], words_to_exclude, w2idx, vectors)

            scores = []
            answer_vectors = []
            for (label, sentence) in group["answers"]:
                # Compute similarity with question vector
                vec = extract_vector(sentence, words_to_exclude, w2idx, vectors) # TODO: Pass these in as args
                answer_vectors.append(vec)
                cosine_distance = sp.spatial.distance.cosine(qvec, vec)
                scores.append((label, cosine_distance))

            # Compute pairwise distances between the answer vectors for K nearest neighbor
            k = min(n_neighbors, len(answer_vectors) - 1) # Minus 1 because have to exclude itself
            # Not enough to do rank propagation. Just keep original scores.
            if k < 0:
#                 print("---> No candidate answers")
                yield None, None
            elif k == 0:
#                 print("---> Not enough candidate answers to do rank propagation. k = {} <---".format(k))
                yield 1, None
            else:                
#                 print("k = {}".format(k))            
                answer_vectors = np.vstack(answer_vectors)
#                 print(answer_vectors.shape)
                W = compute_pairwise_distance_matrix(answer_vectors, k)
                W = np.maximum(W, W.T)  # Ensure W symmetric.
                W[W > 0] = np.exp(- W[W > 0] / (2 * sigma**2))  # Apply gaussian kernel
                D = np.diag(np.sum(W, axis=1))  # Row sum of W
                L = D - W

#                 L = L + eps * np.eye(len(answer_vectors))  # Improve the condition of the graph laplacian
                
                Dinvsqrt = np.sqrt(np.linalg.pinv(D))
                
                # Need to ensure that Dinvsqrt does not have NAN due to division by 0
                assert(not np.any(np.isnan(Dinvsqrt)))
                
                L = Dinvsqrt.dot(L).dot(Dinvsqrt)  # Normalized graph laplacian
                
#                 assert(is_pos_def(Dinvsqrt))
#                 assert(is_pos_def(L))
                
                yield L.shape[0], L
            
            num_questions += 1

---
Load question answer similarity values from file (probably should compute it here)


In [3]:
def load_similarity_features(filepath):
    features = []
    labels = []
    map_label = {"positive": 1, "negative": 0}
    with open(filepath) as infile:
        for line in infile:
            label, score = line.strip().split(',')
            score = float(score)
            label = map_label[label]
            features.append(score)
            labels.append(label)
            
    return np.asarray(features).reshape(-1, 1), np.asarray(labels)

In [4]:
#import training data and test data
train_datapath = "../myclassify/qa.train.arff"
test_datapath = "../myclassify/qa.test.arff"

X_train, y_train = importData(train_datapath)
X_test, y_test = importData(test_datapath)

In [28]:
train_file = "../data/answerSelectionExperiments/data/train-less-than-40.xml"
test_file = "../data/answerSelectionExperiments/data/test-less-than-40.xml"
qn_group_indicators = get_QA_group_indicators(test_file)

Reached EOF at qn 100


---
## Propagate rank score


In [120]:
from cvxpy import Variable, Minimize, norm, quad_form, Problem

In [240]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [325]:
def propagate_scores(r, L, alpha=1.0):
    """Solve convex optimization problem to get new scores"""
        
    n = r.size
    y = Variable(n)
    
    # TODO: Add huber loss
    objective = Minimize( norm(r - y, 1) + alpha * quad_form(y, L) )
#     objective = Minimize( norm(r - y, 1) + alpha * quad_form(y, L) )
    
    constraints = [0 <= y, y <= 1]
    prob = Problem(objective, constraints)

    # The optimal objective is returned by prob.solve().
    result = prob.solve()        
    return y.value.flatten().tolist()[0]

In [324]:
def rank_propagation(data_filepath, r, alpha=1.0, sigma=1.0, n_neighbors=5):
    total_count = 0
    one_count = 0

    # Get qn group indicator
    qn_group_indicators = get_QA_group_indicators(data_filepath)
    qn_number = 0  # Current question number (NOTE: This is not ID in XML)

    scores = []
    for (count, L) in get_weight_matrix(data_filepath, n_neighbors, sigma):
        # Skip question without candidate answers
        if count is None:
            continue

        # Not enough points to propagate. Just use original value.
        if count == 1:
            one_count += 1                
            assert(r[qn_group_indicators == qn_number].size == 1)
            # Append original score
            scores.append(r[qn_group_indicators == qn_number][0])        
        else:
            # Propagate and append new scores
            assert(r[qn_group_indicators == qn_number].size == L.shape[0])
            new_scores = propagate_scores(r[qn_group_indicators == qn_number] , L, alpha)                
            scores += new_scores

        qn_number += 1
        total_count += count

    print("one_count: {} total_count: {}".format(one_count, total_count))

    return np.asarray(scores)

In [312]:
clf_rf = RandomForestClassifier(n_estimators=300, max_depth=12, criterion="gini")
clf_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [346]:
# For testing

# clf = linear_model.LogisticRegression(C=0.005)
# clf.fit(X_train, y_train)

raw_scores = clf.predict_proba(X_test)[:, 1]
# raw_scores = clf_rf.predict_proba(X_test)[:, 1]
# scores = rank_propagation(test_file, raw_scores, alpha=2, sigma=1, n_neighbors=9)
scores = rank_propagation(test_file, raw_scores, alpha=3.0, sigma=1.0, n_neighbors=11)

y_pred_adjusted = (scores >= 0.5)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, scores >= 0.5))
print(accuracy_score(y_test, y_pred))
print(np.sum(np.abs(raw_scores - scores)) / len(scores))  # Average difference between actual
print(np.max(np.abs(raw_scores - scores)))
print(np.sum(y_test != (scores >= 0.5)))
print(np.sum(y_test != (scores >= 0.5)) / float(len(scores)))

print(np.sum(np.abs(y_pred - y_pred_adjusted)))

Reached EOF at qn 95
one_count: 14 total_count: 1517
0.828609096902
0.837837837838
0.00996321110855
0.472836722479
260
0.171390903098
28


In [347]:
P = np.hstack(((1 - scores).reshape(-1, 1), scores.reshape(-1, 1)))
predict_for_test(y_test, y_pred_adjusted, P, "LR_adjusted.txt")

In [348]:
predict_for_test(y_test, y_pred, clf.predict_proba(X_test), "LR.txt")

In [322]:
%pdb off

Automatic pdb calling has been turned OFF


In [275]:
# For training
train_raw_scores = clf.predict_proba(X_train)[:, 1]
scores = rank_propagation(train_file, train_raw_scores, alpha=2, sigma=2, n_neighbors=11)

print(accuracy_score(y_train, scores >= 0.5))
print(np.sum(np.abs(train_raw_scores - scores)) / len(scores))  # Average difference between actual

Reached EOF at qn 93
one_count: 5 total_count: 4718
0.93069097075
0.00652913182896


In [276]:
accuracy_score(y_train, clf.predict(X_train))

0.93535396354387457

In [261]:
y_pred = clf.predict(X_test)

In [262]:
y_pred_adjusted = (scores >= 0.5)
np.sum(y_pred - y_pred_adjusted)

27

In [260]:
251.0 / len(scores)

0.16545814106789716

In [287]:
accuracy_score(y_test, clf.predict(X_test))

0.83783783783783783